# API Data Report
## Paul Keren
### 10/11/2024
The hypothesis that I am testing is if "Happy" by Pharell is the happiest song of all time measured by valence, danceability, and energy. My theoritical application is that the song "Happy" is described as a very happy song in the title and in the lyrics of "Because I'm happy". Though there are many other happy songs that exist in the world so I will be testing the valence, danceability, and energy of each of these tracks to see which one measues the highest. 

The Spotify endpoints I will be using to collect my data are the tracks, features, and albums. The responses I will be using are the valence, energy, and danceability of a track, these respones are suitable for this project because valence measures the musical positiveness in a track so it is a good endpoint to use to test which of the tracks is the most happy. The other respones of danceability and energy are suitable for this hypothesis because if a track is very danceable it means it is upbeat and dancing makes people very happy to be in the moment and having a good time. For energy if a track has a lot of energy it makes the track very optimistic. Some limitations of the data can be user reaction, so we do not know totally how users would react to the top of this list. 

In [33]:
import urllib
import requests
import pandas as pd
import json
import base64

In [34]:
def get_session_token(SessionID, SessionKey):
    url = 'https://accounts.spotify.com/api/token'
    data = {'grant_type':'client_credentials'}
    encoded_key = base64.b64encode(str(SessionID + ":" + SessionKey).\
                                   encode("ascii"))
    header = {'Authorization': 'Basic {}'.format(encoded_key.decode("ascii"))}
    
    response = requests.post(url, 
                            data = data,
                           headers = header)
    print(response.status_code)
    return response.json()['access_token']

In [35]:
keys = pd.read_csv("Keys.txt")

In [36]:
access_token = get_session_token(keys['Client_ID'].iloc[0],keys['Client_Secret'].iloc[0])

200


The access token ^

In [37]:
access_token

'BQDFaQKO1XilzPxs_1z_OkgvFtMTr-B80ZAjKOUQ7sX8UVsPLbask1Y-TcWlagxpA_0s8adKynd_w7pyqBe3zCuBrsnPigz7bH6hsKFkkWocBoM7rF0'

In [38]:
t_features_ep = 'https://api.spotify.com/v1/audio-features/'
tracks_ep = 'https://api.spotify.com/v1/tracks'
ab_tracks_ep = 'https://api.spotify.com/v1/albums/{}/tracks'

The endpoints that are used ^

In [39]:
def api_call(endpoint_url, api_header):
    response = requests.get(endpoint_url, headers = api_header)
    print(response.status_code)
    return response.json()

In [40]:
session_header = {'Authorization': 'Bearer {}'.format(access_token)}

In [41]:
hy_album_id = '0jVGdTKQ0pValbbtxv5h5O'
hy2_album_id = '1FVsTVNu2YGVYcOjY7VtHk'

The albums used are mostly just compilations of generally happy songs or songs that are rated as happy songs, so the variables reflect that being hy = happy, and hy2 = happy 2

In [42]:
ab_tracks_ep.format(hy_album_id)

'https://api.spotify.com/v1/albums/0jVGdTKQ0pValbbtxv5h5O/tracks'

In [43]:
hy_album_response = api_call(ab_tracks_ep.format(hy_album_id) , session_header)
hy2_album_response = api_call(ab_tracks_ep.format(hy2_album_id) , session_header)

200
200


In [44]:
hy_album_df = pd.DataFrame(hy_album_response['items'])
hy2_album_df = pd.DataFrame(hy2_album_response['items']) 
#hy_album_df.head()

In [45]:
hy_track_ids = ','.join(hy_album_df['id'].to_list())
hy2_track_ids = ','.join(hy2_album_df['id'].to_list())

In [46]:
t_features_ep + '?ids={}'.format(hy_track_ids)

'https://api.spotify.com/v1/audio-features/?ids=1huwecVUu6sWgU7o7GHPHQ,2IoU47UbLPu8LX3eAnK7mj,6ZPg2irmu84pBiLUPBSR87,0Ad6w4dXEF0OI3HnZQNuuf,5GtQkJTQ01zxZ9xRuIBRyY,2b80sCsAwCpdWWMdcQYf7s,7fsLtpbscBZUQntENqmR1S,2B9rafHNGPpnaAtqTcbrFL,62tQ11UnK9za7j0dyqT7Hs,6ODJxBvPLPRgKUVFOHClqb,29TUsmgNleSNV7pDAuMpiD,2gEkq0Mrrd6sKUkBnxWqDq,0bP2WhCT0F2pA0vjqaiDSk,0cDbVbHEUM419vxXxigLtS,3IL9TPElS6uk1tgiKEUVie,6TFr1fKugaFyGF7hzH2XQo,6dzH7tWblVkIVRVdFL7sS4,2SMH7dslMqZLscuNIaIsLI,4WPcZxkaJJxAZcbuIVWbhK,51c0YDa7BZUiT79IwDZVkj'

In [47]:
hy_track_features = api_call(t_features_ep + '?ids={}'.format(hy_track_ids), 
                             session_header)
hy2_track_features = api_call(t_features_ep + '?ids={}'.format(hy2_track_ids), 
                             session_header)

200
200


In [48]:
hy_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(hy_track_ids), 
                             session_header)
hy2_track_info = api_call(tracks_ep + '?market=US&ids={}'.format(hy2_track_ids), 
                             session_header)

200
200


In [49]:
hy_features_df = pd.DataFrame(hy_track_features['audio_features'])
hy2_features_df = pd.DataFrame(hy2_track_features['audio_features'])
hy_features_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.647,0.823,5,-4.662,0,0.1840,0.22000,0.000000,0.0908,0.962,160.023,audio_features,1huwecVUu6sWgU7o7GHPHQ,spotify:track:1huwecVUu6sWgU7o7GHPHQ,https://api.spotify.com/v1/tracks/1huwecVUu6sW...,https://api.spotify.com/v1/audio-analysis/1huw...,232720,4
1,0.853,0.681,0,-5.003,1,0.0888,0.00966,0.000057,0.0233,0.930,114.989,audio_features,2IoU47UbLPu8LX3eAnK7mj,spotify:track:2IoU47UbLPu8LX3eAnK7mj,https://api.spotify.com/v1/tracks/2IoU47UbLPu8...,https://api.spotify.com/v1/audio-analysis/2IoU...,270627,4
2,0.652,0.749,0,-5.248,1,0.0886,0.08480,0.000000,0.6040,0.651,94.103,audio_features,6ZPg2irmu84pBiLUPBSR87,spotify:track:6ZPg2irmu84pBiLUPBSR87,https://api.spotify.com/v1/tracks/6ZPg2irmu84p...,https://api.spotify.com/v1/audio-analysis/6ZPg...,267413,4
3,0.637,0.876,5,-5.269,1,0.0366,0.07240,0.000000,0.2150,0.433,116.943,audio_features,0Ad6w4dXEF0OI3HnZQNuuf,spotify:track:0Ad6w4dXEF0OI3HnZQNuuf,https://api.spotify.com/v1/tracks/0Ad6w4dXEF0O...,https://api.spotify.com/v1/audio-analysis/0Ad6...,235733,4
4,0.595,0.858,4,-3.555,0,0.0345,0.02110,0.017500,0.1410,0.737,127.949,audio_features,5GtQkJTQ01zxZ9xRuIBRyY,spotify:track:5GtQkJTQ01zxZ9xRuIBRyY,https://api.spotify.com/v1/tracks/5GtQkJTQ01zx...,https://api.spotify.com/v1/audio-analysis/5GtQ...,222533,4


In [50]:
hy_tracks_df = pd.DataFrame(hy_track_info['tracks'])
hy2_tracks_df = pd.DataFrame(hy2_track_info['tracks'])
hy_tracks_df.head()


,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,preview_url,track_number,type,uri,linked_from
0,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,1,232720,False,{'isrc': 'USQ4E1300686'},{'spotify': 'https://open.spotify.com/track/1h...,https://api.spotify.com/v1/tracks/1huwecVUu6sW...,1huwecVUu6sWgU7o7GHPHQ,False,True,"Happy - From ""Despicable Me 2""",28,https://p.scdn.co/mp3-preview/7df50d6478d9366d...,1,track,spotify:track:1huwecVUu6sWgU7o7GHPHQ,NaN
1,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,1,270626,True,{'isrc': 'GBARL1401524'},{'spotify': 'https://open.spotify.com/track/2I...,https://api.spotify.com/v1/tracks/2IoU47UbLPu8...,2IoU47UbLPu8LX3eAnK7mj,False,True,Uptown Funk (feat. Bruno Mars),28,https://p.scdn.co/mp3-preview/6b518c4b49482770...,2,track,spotify:track:2IoU47UbLPu8LX3eAnK7mj,NaN
2,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,1,267413,False,{'isrc': 'USSM11102908'},{'spotify': 'https://open.spotify.com/track/6Z...,https://api.spotify.com/v1/tracks/6ZPg2irmu84p...,6ZPg2irmu84pBiLUPBSR87,False,True,Love On Top,23,https://p.scdn.co/mp3-preview/b33aaf728bc4a935...,3,track,spotify:track:6ZPg2irmu84pBiLUPBSR87,NaN
3,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,1,235733,False,{'isrc': 'USSM11401744'},{'spotify': 'https://open.spotify.com/track/0A...,https://api.spotify.com/v1/tracks/0Ad6w4dXEF0O...,0Ad6w4dXEF0OI3HnZQNuuf,False,True,Love Never Felt So Good,20,https://p.scdn.co/mp3-preview/84840c0327e036ed...,4,track,spotify:track:0Ad6w4dXEF0OI3HnZQNuuf,NaN
4,"{'album_type': 'compilation', 'artists': [{'ex...",[{'external_urls': {'spotify': 'https://open.s...,1,222533,False,{'isrc': 'GBARL1400296'},{'spotify': 'https://open.spotify.com/track/5G...,https://api.spotify.com/v1/tracks/5GtQkJTQ01zx...,5GtQkJTQ01zxZ9xRuIBRyY,False,True,Summer,32,https://p.scdn.co/mp3-preview/f8877a2ec4963cb1...,5,track,spotify:track:5GtQkJTQ01zxZ9xRuIBRyY,NaN


We start getting good data here but we need to wrap it up more

In [51]:
hy_merged = pd.merge(hy_features_df, hy_tracks_df,
         how = 'inner', on = 'id')
hy_merged.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,href,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y,linked_from
0,0.647,0.823,5,-4.662,0,0.1840,0.22000,0.000000,0.0908,0.962,...,https://api.spotify.com/v1/tracks/1huwecVUu6sW...,False,True,"Happy - From ""Despicable Me 2""",28,https://p.scdn.co/mp3-preview/7df50d6478d9366d...,1,track,spotify:track:1huwecVUu6sWgU7o7GHPHQ,NaN
1,0.853,0.681,0,-5.003,1,0.0888,0.00966,0.000057,0.0233,0.930,...,https://api.spotify.com/v1/tracks/2IoU47UbLPu8...,False,True,Uptown Funk (feat. Bruno Mars),28,https://p.scdn.co/mp3-preview/6b518c4b49482770...,2,track,spotify:track:2IoU47UbLPu8LX3eAnK7mj,NaN
2,0.652,0.749,0,-5.248,1,0.0886,0.08480,0.000000,0.6040,0.651,...,https://api.spotify.com/v1/tracks/6ZPg2irmu84p...,False,True,Love On Top,23,https://p.scdn.co/mp3-preview/b33aaf728bc4a935...,3,track,spotify:track:6ZPg2irmu84pBiLUPBSR87,NaN
3,0.637,0.876,5,-5.269,1,0.0366,0.07240,0.000000,0.2150,0.433,...,https://api.spotify.com/v1/tracks/0Ad6w4dXEF0O...,False,True,Love Never Felt So Good,20,https://p.scdn.co/mp3-preview/84840c0327e036ed...,4,track,spotify:track:0Ad6w4dXEF0OI3HnZQNuuf,NaN
4,0.595,0.858,4,-3.555,0,0.0345,0.02110,0.017500,0.1410,0.737,...,https://api.spotify.com/v1/tracks/5GtQkJTQ01zx...,False,True,Summer,32,https://p.scdn.co/mp3-preview/f8877a2ec4963cb1...,5,track,spotify:track:5GtQkJTQ01zxZ9xRuIBRyY,NaN


More good data as we can see the variables we are going to measure

In [52]:
hy_merged.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type_x', 'id', 'uri_x', 'track_href', 'analysis_url', 'duration_ms_x',
       'time_signature', 'album', 'artists', 'disc_number', 'duration_ms_y',
       'explicit', 'external_ids', 'external_urls', 'href', 'is_local',
       'is_playable', 'name', 'popularity', 'preview_url', 'track_number',
       'type_y', 'uri_y', 'linked_from'],
      dtype='object')

In [53]:
hy2_merged = pd.merge(hy2_features_df, hy2_tracks_df,
                     how = 'inner', on = 'id')
hy_merged['artist'] = "Various Artists"
hy2_merged['artist'] = "Multiple Artists"
hy2_merged.shape

(20, 34)

Again the albums are compilations so there is no specific artist for each song, hence why the artist name is Various and Multiple

In [54]:
artist_compare = pd.concat([hy_merged, hy2_merged], axis = 0)
artist_compare

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,is_local,is_playable,name,popularity,preview_url,track_number,type_y,uri_y,linked_from,artist
0,0.647,0.823,5,-4.662,0,0.1840,0.220000,0.000000,0.0908,0.962,...,False,True,"Happy - From ""Despicable Me 2""",28,https://p.scdn.co/mp3-preview/7df50d6478d9366d...,1,track,spotify:track:1huwecVUu6sWgU7o7GHPHQ,NaN,Various Artists
1,0.853,0.681,0,-5.003,1,0.0888,0.009660,0.000057,0.0233,0.930,...,False,True,Uptown Funk (feat. Bruno Mars),28,https://p.scdn.co/mp3-preview/6b518c4b49482770...,2,track,spotify:track:2IoU47UbLPu8LX3eAnK7mj,NaN,Various Artists
2,0.652,0.749,0,-5.248,1,0.0886,0.084800,0.000000,0.6040,0.651,...,False,True,Love On Top,23,https://p.scdn.co/mp3-preview/b33aaf728bc4a935...,3,track,spotify:track:6ZPg2irmu84pBiLUPBSR87,NaN,Various Artists
3,0.637,0.876,5,-5.269,1,0.0366,0.072400,0.000000,0.2150,0.433,...,False,True,Love Never Felt So Good,20,https://p.scdn.co/mp3-preview/84840c0327e036ed...,4,track,spotify:track:0Ad6w4dXEF0OI3HnZQNuuf,NaN,Various Artists
4,0.595,0.858,4,-3.555,0,0.0345,0.021100,0.017500,0.1410,0.737,...,False,True,Summer,32,https://p.scdn.co/mp3-preview/f8877a2ec4963cb1...,5,track,spotify:track:5GtQkJTQ01zxZ9xRuIBRyY,NaN,Various Artists
5,0.606,0.720,5,-6.253,0,0.0283,0.000149,0.001650,0.0745,0.178,...,False,True,All of Me - Tiësto's Birthday Treatment Remix ...,19,https://p.scdn.co/mp3-preview/976862b21ada9ab2...,6,track,spotify:track:2b80sCsAwCpdWWMdcQYf7s,NaN,Various Artists
6,0.654,0.868,1,-3.053,1,0.0458,0.025600,0.000000,0.0766,0.542,...,False,True,Best Song Ever,20,https://p.scdn.co/mp3-preview/41868c3ce389ff76...,7,track,spotify:track:7fsLtpbscBZUQntENqmR1S,NaN,Various Artists
7,0.811,0.879,9,-3.724,1,0.0513,0.047600,0.000002,0.1090,0.962,...,False,True,All About That Bass,18,https://p.scdn.co/mp3-preview/9517f6a04eaa56dd...,8,track,spotify:track:2B9rafHNGPpnaAtqTcbrFL,NaN,Various Artists
8,0.700,0.747,5,-3.453,1,0.0964,0.274000,0.000000,0.4860,0.683,...,False,True,Bills,20,https://p.scdn.co/mp3-preview/5841a9ea6571e400...,9,track,spotify:track:62tQ11UnK9za7j0dyqT7Hs,NaN,Various Artists
9,0.827,0.580,4,-7.793,1,0.0374,0.179000,0.030100,0.0699,0.381,...,False,True,Cheerleader (Felix Jaehn Remix) - Radio Edit,19,https://p.scdn.co/mp3-preview/b18fa4a865ff7b43...,10,track,spotify:track:6ODJxBvPLPRgKUVFOHClqb,NaN,Various Artists


In [55]:
name_valence_energy_danceability = hy_merged[["name", "valence", "energy", "danceability"]]

In [56]:
name_valence_energy_danceability.head(30)

,name,valence,energy,danceability
0,"Happy - From ""Despicable Me 2""",0.962,0.823,0.647
1,Uptown Funk (feat. Bruno Mars),0.930,0.681,0.853
2,Love On Top,0.651,0.749,0.652
3,Love Never Felt So Good,0.433,0.876,0.637
4,Summer,0.737,0.858,0.595
5,All of Me - Tiësto's Birthday Treatment Remix ...,0.178,0.720,0.606
6,Best Song Ever,0.542,0.868,0.654
7,All About That Bass,0.962,0.879,0.811
8,Bills,0.683,0.747,0.700
9,Cheerleader (Felix Jaehn Remix) - Radio Edit,0.381,0.580,0.827


Using our responses that we are highlighting for the hypothesis and putting them in a list of their own

In [57]:
name_valence_energy_danceability = hy2_merged[["name", "valence", "energy", "danceability"]]

In [58]:
name_valence_energy_danceability.head(30)

,name,valence,energy,danceability
0,Here It Goes Again,0.810,0.882,0.537
1,How Bizarre,0.797,0.748,0.749
2,Superstition,0.877,0.583,0.611
3,It's Not Unusual,0.950,0.815,0.521
4,5 Years Time,0.749,0.552,0.804
5,I Got You (I Feel Good),0.705,0.522,0.603
6,Come And Get It,0.776,0.727,0.649
7,Take Your Mama,0.937,0.948,0.762
8,Funkytown - Single Version,0.315,0.591,0.915
9,I Will Survive - Single Version,0.738,0.810,0.724


The conclusion we can draw from this data is that "All About that Bass" by Meghan Trainor is regarded as the happiest song based on the average of valence, energy, and danceability. "All About that Bass" scored an average of 0.884, while coming in second was "Take Your Mama" with an average of 0.882. The top five rolls out with "Super Freak" by Rick James with a score of 0.866, "Uptown Funk" by Mark Ronson and Bruno Mars at fourth with an average of 0.821 and finally, the song I hypothesized would be the happiest song, "Happy" by Pharrell came in fifth with a score of 0.811. 

Some limitations of this data can be that user reaction is not shown in these charts as this is just data from Spotify itself on the dimensions of the track and not people's actual feelings about a track. An alternative approache could be creating a poll or something for individuals to vote on for them to decide on what song makes them the happiest, a next step could be propsing to data to people and asking them if they agreed with my findings or they feel some songs should be moved around.  